In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM
from sklearn.base import BaseEstimator, RegressorMixin
from keras.models import Sequential
from tensorflow.keras.models import Model
from keras.layers import Input, GRU, LSTM, Dense, Dropout, Concatenate, Reshape
from keras.optimizers import Adam
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

In [21]:
test_data = pd.read_csv('test_stn+ef_label.csv')
test_data.head()

,stn4contest,ef_datetime,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,class_interval
0,31,2023-05-01 12:00:00,68.0,46.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,12,NaN
1,31,2023-05-01 15:00:00,83.0,66.0,26.0,5.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,15,NaN
2,31,2023-05-01 18:00:00,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,18,NaN
3,31,2023-05-01 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,21,NaN
4,31,2023-05-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,2,0,NaN


In [5]:
import pickle

# 모델이 저장된 파일 경로
filename = 'cat_best_model.pkl'

# 피클 파일에서 모델 불러오기
with open(filename, 'rb') as file:
    cat_model = pickle.load(file)

In [8]:
# 모델이 저장된 파일 경로
filename = 'rf_best_model.pkl'

# 피클 파일에서 모델 불러오기
with open(filename, 'rb') as file:
    rf_model = pickle.load(file)

In [14]:
# 모델이 저장된 파일 경로
filename = 'lgb_best_model.pkl'

# 피클 파일에서 모델 불러오기
with open(filename, 'rb') as file:
    lgb_model = pickle.load(file)

In [22]:
features = test_data[['ef_datetime', 'stn4contest', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09', 'year', 'month', 'day', 'hour']]
target = test_data['class_interval']

features.set_index('ef_datetime', inplace=True)

# 데이터 정렬
features = features.sort_index()

# 데이터 정규화
scaler_features = MinMaxScaler(feature_range=(0, 1))

test_features_scaled = scaler_features.fit_transform(features)

In [23]:
# 예측
cat_test_predict = cat_model.predict(test_features_scaled)
lgb_test_predict = lgb_model.predict(test_features_scaled)
rf_test_predict = rf_model.predict(test_features_scaled)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


In [24]:
test_predict = (cat_test_predict+lgb_test_predict+rf_test_predict)/3
predicted_df = pd.DataFrame(test_predict, index=features.index, columns=['Predicted vv'])

In [25]:
def get_class_interval(rainfall):
    if rainfall < 0.1:
        return 0
    elif 0.1 <= rainfall < 0.2:
        return 1
    elif 0.2 <= rainfall < 0.5:
        return 2
    elif 0.5 <= rainfall < 1.0:
        return 3
    elif 1.0 <= rainfall < 2.0:
        return 4
    elif 2.0 <= rainfall < 5.0:
        return 5
    elif 5.0 <= rainfall < 10.0:
        return 6
    elif 10.0 <= rainfall < 20.0:
        return 7
    elif 20.0 <= rainfall < 30.0:
        return 8
    else:
        return 9


predicted_df['class_interval'] = predicted_df['Predicted vv'].apply(get_class_interval)
predicted_df['class_interval'].value_counts()

class_interval
0    2131
1    1711
2     623
5     459
6     417
3     387
4     345
7     338
8      75
9      14
Name: count, dtype: int64

In [27]:
# 'Predicted vv' 열에 변환 함수 적용하여 클래스 구간 계산
predicted_df['Class Interval'] = predicted_df['Predicted vv'].apply(get_class_interval)

# 원래의 test_data에 'Class Interval' 열 추가
test_data['class_interval'] = predicted_df['Class Interval'].values
test_data.head()

,stn4contest,ef_datetime,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,class_interval
0,31,2023-05-01 12:00:00,68.0,46.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,12,2
1,31,2023-05-01 15:00:00,83.0,66.0,26.0,5.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,15,0
2,31,2023-05-01 18:00:00,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,18,0
3,31,2023-05-01 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,21,1
4,31,2023-05-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,2,0,1


In [30]:
submit = pd.read_csv("rainfall_test2.csv")
realsubmit = pd.read_csv("rainfall_test.csv")
prediction_test = test_data.copy()

#submit-prediction 지점,date 맞추기
prediction_test['ef_datetime'] = pd.to_datetime(prediction_test['ef_datetime'])
submit['ef_year_temp'] = submit['ef_year'].replace({'D': '2023'})
submit['ef_datetime'] = pd.to_datetime(submit[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')
stn_mapping = {'STN031': 31, 'STN032': 32, 'STN033': 33, 'STN034': 34, 'STN035': 35}
submit['stn4contest'] = submit['stn4contest'].replace(stn_mapping)
    
merged = submit.merge(prediction_test[['ef_datetime', 'stn4contest', 'class_interval']], 
                      on=['ef_datetime', 'stn4contest'], 
                      how='left', 
                      suffixes=('', '_pred'))
merged['class_interval_pred']
    
#-999 index 저장
idx999 = realsubmit[realsubmit['rainfall_test.class_interval'].notnull()].index
merged.loc[idx999, 'class_interval_pred'] = -999

# 제출용 데이터에 class_interval 채우기
realsubmit['rainfall_test.class_interval'] = merged['class_interval_pred']

#저장
realsubmit.to_csv("240494.csv")  
print(realsubmit.shape)
realsubmit.head()

(122000, 20)


,rainfall_test.fc_year,rainfall_test.fc_month,rainfall_test.fc_day,rainfall_test.fc_hour,rainfall_test.stn4contest,rainfall_test.dh,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,rainfall_test.ef_hour,rainfall_test.v01,rainfall_test.v02,rainfall_test.v03,rainfall_test.v04,rainfall_test.v05,rainfall_test.v06,rainfall_test.v07,rainfall_test.v08,rainfall_test.v09,rainfall_test.class_interval
0,D,5,1,9,STN031,3,D,5,1,12,68,46,13,1,0,0,0,0,0,2
1,D,5,1,9,STN031,6,D,5,1,15,83,66,26,5,0,0,0,0,0,0
2,D,5,1,9,STN031,9,D,5,1,18,17,6,0,0,0,0,0,0,0,0
3,D,5,1,9,STN031,12,D,5,1,21,0,0,0,0,0,0,0,0,0,1
4,D,5,1,9,STN031,15,D,5,2,0,0,0,0,0,0,0,0,0,0,1
